In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

In [ ]:
ruta_bd = r"C:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb"
arcpy.env.workspace = ruta_bd

capa_riego_rural = ['Riesgo_Urbano_Inundacion','Riesgo_Rural_Incendio','Riesgo_Rural_Remocion_Masa']
campos_actualizar = ['Riesgo']
nombre_dataset = ['Analisis_Vulnerabilidad_Riesgo_Inundacion',
           'Analisis_Vulnerabilidad_Riesgo_Incendio',
           'Analisis_Vulnerabilidad_Riesgo_Remocion_Masa']

for dataset in arcpy.ListDatasets():
    for seleccion_dataset in nombre_dataset:
        if dataset == seleccion_dataset:
            for capa in arcpy.ListFeatureClasses(feature_dataset=dataset):
                for seleccion_capa in capa_riego_rural:
                    if capa == seleccion_capa:
                        with arcpy.da.UpdateCursor(capa, campos_actualizar) as cursor:
                            for fila in cursor:
                                if fila[0] == 'Muy Alto':
                                    fila[0] = 'Muy alto'
                                elif fila[0] == 'Muy Bajo':
                                    fila[0] = 'Muy bajo'
                                elif fila[0] == 'Alta':
                                    fila[0] = 'Alto'
                                elif fila[0] == 'Baja':
                                    fila[0] = 'Bajo'
                                elif fila[0] == 'Media':
                                    fila[0] = 'Medio'
                                elif fila[0] == 'Riesgo Alto':
                                    fila[0] = 'Alto'
                                elif fila[0] == 'Riesgo Medio':
                                    fila[0] = 'Medio'
                                elif fila[0] == 'Riesgo Bajo':
                                    fila[0] = 'Bajo'
                                elif fila[0] == None:
                                    fila[0] = 'Sin información'
                                else:
                                    print("No se ejecuta cambio")                            
                                # ? Se actualiza el cursor con la actualización de la lista anterior
                                cursor.updateRow(fila)

In [ ]:
# Generación de DF
df_variables_economicas_cundinamarca = pd.read_csv("Variables_Economicas_Cundinamarca_DANE.csv", sep=";")

# !! Primer Tratamiento
df_variables_economicas_cundinamarca['codigo_municipio_dane'] = df_variables_economicas_cundinamarca.loc[:,['codigo_municipio_dane']].astype('str')

# Migración a Feature Table
nombre_tabla_variables_economicas = "tbl_variables_economicas_dane"
ruta_salida_variables_economicas = os.path.join(arcpy.env.workspace, nombre_tabla_variables_economicas)
df_variables_economicas_cundinamarca.spatial.to_table(location = ruta_salida_variables_economicas)

# Creación de la relación entre Municipio Vs Variables Económicas
r_municipio_variables = "r_municipio_variables"
capa_municipio = "municipio"

# ********************** Relación Capa - Temática **********************
ruta_municipio = os.path.join(arcpy.env.workspace, capa_municipio)

arcpy.management.CreateRelationshipClass(ruta_municipio, ruta_salida_variables_economicas, r_municipio_variables, 'SIMPLE', 'variable', 'municipio', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_municipio_dane', 'codigo_municipio_dane')
print("Creada relación: {0}".format(r_municipio_variables))

<h4>Población CAR</h4>

In [ ]:
df_poblacion_car = pd.read_csv("poblacion_car.csv", sep=";")

# !! Tratamiento
df_poblacion_car['codigo_municipio'] = df_poblacion_car.loc[:,['codigo_municipio']].astype('str')

# Migración a Feature Table
nombre_poblacion_car = "tbl_poblacion_car"
ruta_salida_poblacion_car = os.path.join(arcpy.env.workspace, nombre_poblacion_car)
df_poblacion_car.spatial.to_table(location = ruta_salida_poblacion_car)

# Creación de la relación entre Municipio Vs Variables Económicas
r_municipio_poblacion = "r_municipio_poblacion"

# ********************** Relación Capa - Temática **********************
ruta_municipio = os.path.join(arcpy.env.workspace, capa_municipio)

arcpy.management.CreateRelationshipClass(ruta_municipio, ruta_salida_poblacion_car, r_municipio_poblacion, 'SIMPLE', 'poblacion', 'municipio', 
                                         "NONE", 'ONE_TO_ONE', "NONE", 'codigo_municipio_dane', 'codigo_municipio')
print("Creada relación: {0}".format(r_municipio_poblacion))